In [8]:
import os
import numpy as np
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
import pypianoroll as ppr

In [9]:
def write_midi(filepath, pianorolls, config):
    is_drums = config['is_drums']
    track_names = config['track_names']
    tempo = config['tempo']
    beat_resolution = config['beat_resolution']
    
    program_nums = None
    
    if not np.issubdtype(pianorolls.dtype, np.bool_):
        raise TypeError("Support only binary-valued piano-rolls")
    if isinstance(program_nums, int):
        program_nums = [program_nums]
    if isinstance(is_drums, int):
        is_drums = [is_drums]

    if program_nums is None:
        program_nums = [0] * len(pianorolls)
    if is_drums is None:
        is_drums = [False] * len(pianorolls)

    multitrack = Multitrack(beat_resolution=beat_resolution, tempo=tempo)
    for idx in range(pianorolls.shape[2]):
        if track_names is None:
            track = Track(pianorolls[..., idx], program_nums[idx],
                          is_drums[idx])
        else:
            track = Track(pianorolls[..., idx], program_nums[idx],
                          is_drums[idx], track_names[idx])
        multitrack.append_track(track)
    multitrack.write(filepath)

In [10]:
def save_midi(filepath, phrases, config):
    if not np.issubdtype(phrases.dtype, np.bool_):
        raise TypeError("Support only binary-valued piano-rolls")

    reshaped = phrases.reshape(-1, phrases.shape[1] * phrases.shape[2],
                               phrases.shape[3], phrases.shape[4])
    
    print("reshaped shape:", reshaped.shape)
    
    pad_width = ((0, 0), (0, config['pause_between_samples']),
                 (config['lowest_pitch'],
                  128 - config['lowest_pitch'] - config['num_pitch']),
                 (0, 0))
    
    # pad width 表示前补和后补的长度
    # print('pad_width:',pad_width)
    padded = np.pad(reshaped, pad_width, 'constant')
    # print("padded shape:", padded.shape)
    pianorolls = padded.reshape(-1, padded.shape[2], padded.shape[3])
    # print("pianorolls shape:", pianorolls.shape)
    write_midi(filepath, pianorolls, config)

In [11]:
config_8b = {
    # Music
    'track_num': 8,
    'programs': [0, 0, 24, 32, 48, 64, 80, 88],
    'is_drums': [True, False, False, False, False, False, False, False],
    'track_names': ['Drums', 'Piano', 'Guitar', 'Bass', 'Ensemble',
                    'Reed', 'Synth Lead','Synth Pad'],
    'tempo': 120,
    'velocity': 100,
    # Dataset
    'dataset_path': '/Users/mac/Desktop/Brain/MuseGAN/training_data/lastfm_alternative_8b_phrase.npy',
    'dataset_name': 'lastfm_alternative_8b_phrase',
    # Playback
    'pause_between_samples': 96,
    # Data
    'num_bar': 4,
    'num_beat': 4,
    'num_pitch': 84,
    'num_track': 8,
    'num_timestep': 96,
    'beat_resolution': 24,
    'lowest_pitch': 24
    # MIDI note number of the lowest pitch in data tensors
}

In [12]:
config_5b = {
    # Music
    'track_num': 5,
    'programs': [0, 0, 24, 32, 48],
    'is_drums': [True, False, False, False, False],
    'track_names': ['Drums', 'Piano', 'Guitar', 'Bass', 'Ensemble'],
    'tempo': 120,
    'velocity': 100,
    # Dataset
    'dataset_path': '/Users/mac/Desktop/Brain/MuseGAN/training_data/lastfm_alternative_5b_phrase.npy',
    'dataset_name': 'lastfm_alternative_5b_phrase',
    # Playback
    'pause_between_samples': 96,
    # Data
    'num_bar': 4,
    'num_beat': 4,
    'num_pitch': 84,
    'num_track': 8,
    'num_timestep': 96,
    'beat_resolution': 24,
    'lowest_pitch': 24
    # MIDI note number of the lowest pitch in data tensors
}

In [13]:
def ith_song_to_midi(dataset, index, midi_dir, config):
    now_song = dataset[index]
    midi_name = config['dataset_name']+'_'+str(index)+'.mid'
    midi_path = os.path.join(midi_dir, midi_name)
    save_midi(midi_path, now_song, config)
    print("Save %s to %s" % (midi_name, midi_path))

In [14]:
midi_dir = '/Users/mac/Desktop/test_ppr/'
config = config_5b
dataset = np.load(config['dataset_path'])
for i in range(20):
    ith_song_to_midi(dataset, i, midi_dir, config)

reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_0.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_0.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_1.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_1.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_2.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_2.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_3.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_3.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_4.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_4.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_5.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_5.mid
reshaped shape: (6, 384, 84, 5)
Save lastfm_alternative_5b_phrase_6.mid to /Users/mac/Desktop/test_ppr/lastfm_alternative_5b_phrase_6.mid
reshaped shape: (6, 384, 84, 5)
Sa